In [1]:
pip install boardgamegeek2

In [3]:
import json
import csv

#BGG API - https://github.com/lcosmin/boardgamegeek  /  https://lcosmin.github.io/boardgamegeek/
from boardgamegeek import BGGClient
bgg = BGGClient()


In [96]:
#Creating lookup dict of game id (string) : game title (string) to convert between the two
game_id = []
game_title = []
with open('data/game_lookup.csv', encoding = 'utf8') as infile:
    reader = csv.reader(infile, delimiter=',')
    for row in reader:        
        game_id.append(row[0])
        game_title.append(row[1])
game_lookup_dict = {game_id[i]: game_title[i] for i in range(len(game_id))}
#game_lookup_dict

#Creating lookup dict of game title (string) : game id (string) for the 60 games in the questionnaire
game_id = []
game_title = []
with open('data/survey_60games.csv', encoding = 'utf8') as infile:
    reader = csv.reader(infile, delimiter=',')
    for row in reader:                
        game_title.append(row[0])
        game_id.append(row[1])
quest_games_lookup_dict = {game_title[i]: game_id[i] for i in range(len(game_id))}
#quest_games_lookup_dict

In [16]:
#Survey #1: BGG username survey (the short one)
#Just for testing webhook JSON responses, in practice we will use Flask to ingest webhook responses
with open('data/bgg_survey_webhook.json') as fp:
    bgg_survey_json = json.load(fp)
    
"""
Structure of Survey #1: BGG username integration
Four questions:
0. What is your BGG username? (string)
1. Target age range (5 choices)
2. Number of players (4 choices)
3. Target playing time (5 choices)
"""
print('Q0 POSSIBLE VALUES:')
print('string - BGG username')
print('\nQ1 POSSIBLE VALUES:')
for i in bgg_survey_json['body']['form_response']['definition']['fields'][1]['choices']:
  print(i['label'])

print('\nQ2 POSSIBLE VALUES:')
for i in bgg_survey_json['body']['form_response']['definition']['fields'][2]['choices']:
  print(i['label'])

print('\nQ3 POSSIBLE VALUES:')
for i in bgg_survey_json['body']['form_response']['definition']['fields'][3]['choices']:
  print(i['label'])

Q0 POSSIBLE VALUES:
string - BGG username

Q1 POSSIBLE VALUES:
Toddler (2-5 years old)
Children (6-11 years old)
Youth (12-16 years old)
Adult (17+ years old)
Any

Q2 POSSIBLE VALUES:
Solo or Pair (1-2 players)
Small Group (2-4 players)
Party (5+ players)
Any

Q3 POSSIBLE VALUES:
Short (5-30 minutes)
Medium (30-60 minutes)
Long (60-120 minutes)
Multiday (120+ minutes)
Any


In [31]:
#Survey #2: Game questionnaire (long one)
#For testing webhook JSON responses, in practice we will use Flask to ingest webhook responses
with open('data/questionnaire_webhook.json') as fp:
    questionnaire_json = json.load(fp)
    
"""
Structure of Survey #2: Game Questionnaire
Nine questions:
0. Target age range (5 choices)
1. Number of players (4 choices)
2. Target playing time (5 choices)
3. Favorite game 1/6 (11 choices each)
4. Favorite game 2/6
5. Favorite game 3/6
6. Favorite game 4/6
7. Favorite game 5/6
8. Favorite game 6/6
"""
#Possible values:
for i in range (9):
  print('\nQ' + str(i) + ' POSSIBLE VALUES:')
  for c in questionnaire_json['body']['form_response']['definition']['fields'][i]['choices']:
    print(c['label'])


Q0 POSSIBLE VALUES:
Toddler (2-5 years old)
Children (6-11 years old)
Youth (12-16 years old)
Adult (17+ years old)
Any

Q1 POSSIBLE VALUES:
Solo or Pair (1-2 players)
Small Group (2-4 players)
Party (5+ players)
Any

Q2 POSSIBLE VALUES:
Short (5-30 minutes)
Medium (30-60 minutes)
Long (60-120 minutes)
Multiday (120+ minutes)
Any

Q3 POSSIBLE VALUES:
7 Wonders
Arkham Horror
Azul
Battleship
Chutes and Ladders
Codenames
Exploding Kittens
Othello
Power Grid
Star Wars: X-Wing Miniatures
I don't like any of these games

Q4 POSSIBLE VALUES:
Apples to Apples
Backgammon
Battlestar Galactica
Dixit
Gloomhaven
Loopin' Louie
Operation
Sushi Go!
Ticket to Ride
Star Wars: X-Wing Miniatures
I don't like any of these games

Q5 POSSIBLE VALUES:
Blokus
Connect Four
Coup
Dominion
Magic: The Gathering
Memoir '44
Munchkin
Risk
Sleeping Queens
Small World
I don't like any of these games

Q6 POSSIBLE VALUES:
Arkham Horror: The Card Game
Go
Monopoly
Pandemic
Pandemic Legacy S1
Root
Sorry!
Splendor
Terraformi

In [91]:
#Functions to pull answers from the webhook JSONs
"""
Returns a dict with the min and max age that games should be filtered on
Dict format: 3 fields
    - question: the question being answered [age]
    - min: minimum age to use in filter
    - max: maximum age to use in filter
"""
def player_age_response(text):
    d = {"question" : "age"}
    if text == 'Toddler (2-5 years old)':
        d['min'] = 2
        d['max'] = 5
    elif text == 'Children (6-11 years old)':
        d['min'] = 6
        d['max'] = 11
    elif text == 'Youth (12-16 years old)':
        d['min'] = 10
        d['max'] = 16
    elif text == 'Adult (17+ years old)':
        d['min'] = 12
        d['max'] = 100
    else:
        d['min'] = 2
        d['max'] = 100
    return d
    
"""
Returns a dict with the min and max players that games should be filtered on
Dict format: 3 fields
    - question: the question being answered [num_players]
    - min: minimum number of players to use in filter
    - max: maximum number of players to use in filter
"""
def player_num_response(text):
    d = {"question" : "num_players"}
    if text == 'Solo or Pair (1-2 players)':
        d['min'] = 1
        d['max'] = 2
    elif text == 'Small Group (2-4 players)':
        d['min'] = 2
        d['max'] = 4
    elif text == 'Party (5+ players)':
        d['min'] = 5
        d['max'] = 100  
    else:
        d['min'] = 1
        d['max'] = 100
    return d        

"""
Returns a dict with the min and max playing time in minutes that games should be filtered on
Dict format: 3 fields
    - question: the question being answered [play_time]
    - min: minimum number of minutes of expected playtime to use in filter
    - max: maximum number of minutes of expected playtime to use in filter
"""
def player_time_response(text):
    d = {"question" : "play_time"}
    if text == 'Short (5-30 minutes)':
        d['min'] = 5
        d['max'] = 30
    elif text == 'Medium (30-60 minutes)':
        d['min'] = 30
        d['max'] = 60
    elif text == 'Long (60-120 minutes)':
        d['min'] = 60
        d['max'] = 120  
    elif text == 'Multiday (120+ minutes)':
        d['min'] = 120
        d['max'] = 100000  
    else:
        d['min'] = 5
        d['max'] = 100000
    return d     

"""
How to pull answers from Survey #1
Input: json file produced by the BGG Survey webhook
Returns: dict of responses with the following fields:
    - survey: name of survey (bgg_user)
    - user_id: user id (string) for BoardGameGeek.com
    - age: dict of age filters produced by player_age_response
    - num_players: dict of player number filters produced by player_num_response
    - play_time: dict of playing time filters produced by player_time_response
    - games: dict of games the player likes in the form of game_id : game_name (both strings)
"""
def get_bgg_survey_answers(json_data):
    d = {"survey" : "bgg_user"}
    user_id = json_data['body']['form_response']['answers'][0]['text']    
    d['user_id'] = user_id    
    d['age'] = player_age_response(json_data['body']['form_response']['answers'][1]['choice']['label'])
    d['num_players'] = player_num_response(json_data['body']['form_response']['answers'][2]['choice']['label'])
    d['play_time'] = player_time_response(json_data['body']['form_response']['answers'][3]['choice']['label'])    
    #Pull game collection using BGG API
    collection = bgg.collection(user_id, min_rating=7.0)
    game_list = {}
    for i in collection.items:
        game_list[str(i.id)] = i.name
    d['games'] = game_list
    return d

"""
How to pull answers from Survey #2
Input: json file produced by the Game Questionnaire webhook
Returns: dict of responses with the following fields:
    - survey: name of survey [questionnaire]    
    - age: dict of age filters produced by player_age_response
    - num_players: dict of player number filters produced by player_num_response
    - play_time: dict of playing time filters produced by player_time_response
    - games: dict of games the player likes in the form of game_id : game_name (both strings)
"""
def get_quest_survey_answers(json_data):
    d = {"survey" : "questionnaire"}    
    d['age'] = player_age_response(json_data['body']['form_response']['answers'][0]['choice']['label'])
    d['num_players'] = player_num_response(json_data['body']['form_response']['answers'][1]['choice']['label'])
    d['play_time'] = player_time_response(json_data['body']['form_response']['answers'][2]['choice']['label'])    
    game_list = {}
    for i in range(3, 9):
        game_title = json_data['body']['form_response']['answers'][i]['choice']['label']
        game_id = quest_games_lookup_dict[game_title]
        #Exclude if they don't like any of the games, code '99999999'
        if game_id != '99999999':
            game_list[game_id] = game_title
    d['games'] = game_list
    return d

In [95]:
print(get_bgg_survey_answers(bgg_survey_json))
print(get_quest_survey_answers(questionnaire_json))

{'survey': 'bgg_user', 'user_id': 'Stupendous12', 'age': {'question': 'age', 'min': 6, 'max': 11}, 'num_players': {'question': 'num_players', 'min': 2, 'max': 4}, 'play_time': {'question': 'play_time', 'min': 30, 'max': 60}, 'games': {'13': 'Catan', '133473': 'Sushi Go!', '9209': 'Ticket to Ride'}}
{'survey': 'questionnaire', 'age': {'question': 'age', 'min': 10, 'max': 16}, 'num_players': {'question': 'num_players', 'min': 5, 'max': 100}, 'play_time': {'question': 'play_time', 'min': 5, 'max': 100000}, 'games': {'230802': 'Azul', '9209': 'Ticket to Ride', '36218': 'Dominion', '30549': 'Pandemic', '31260': 'Agricola'}}
